EDAF75 - notebook for lecture 2
===============================

**Author:** Christian Söderberg



## Week 1: Introduction, relational databases, and SQL, continued&#x2026;



In order to run SQL in this notebook, we need to load the SQL extension:



In [1]:
%load_ext sql

&#x2026; and then we want some database to work on (I've put all the databases used for this lecture in a file called `lect01.sqlite`):



In [2]:
%sql sqlite:///lect01.sqlite

Now we're good to go, we just have to prefix our SQL queries with `%sql` (one line of SQL) or `%%sql` (several lines of SQL, this is the form we will use in most cases).



### `GROUP BY` and `HAVING`



We talked about `GROUP BY` and `HAVING` last time, but I skipped the following section.



#### Using `CASE WHEN`



When we group values, we can make use of the `CASE WHEN` construct (see the penultimate track in the diagram below):

<center>
<img src="./select-expr.svg">
</center>

For instance, we can use it to categorize the era for each of the physics laureates having a name beginning with 'A':



In [1]:
%%sql
SELECT   year, name,
         CASE
             WHEN year < 1970 THEN 'ancient time'
             WHEN year <= 2000 THEN 'a long time ago'
             ELSE 'quite recently'
         END AS era
FROM     nobel_prizes
WHERE    category = 'physics' AND name LIKE 'A%'

**Problem:** Use the `CASE WHEN` construct and `GROUP BY` to count the number of physics laureates beginning with 'A' in each era. Since we've now seen CTEs, we can try using one here.



In [1]:
%%sql

### A short intro to window functions



As we saw above, we can treat partitions of our rows as 'groups', by using `GROUP BY`.

We also saw than when we use an aggregate function on a group, we collapse whole groups into one row in the output (and when we apply it to a table without groups, the whole table collapses) &#x2013; but sometimes we want to apply (aggregate) functions within a group of values, and keep each row in the output.

Above we listed all laureates in 2013, now we want to add one column to the output: for each laureate, we want to show how many laureates shared the prize in her or his category.

If we use `GROUP BY` and the aggregate function `count()` on the categories, we would only get one row per category, and using `count()` without grouping would collapse the whole result into just one row:



In [1]:
%%sql
SELECT    year, category, name, count() AS count      -- oh no!
FROM      nobel_prizes
WHERE     year = 2013
ORDER BY  category

Fortunately, SQL has quite recently introduced a way to apply functions only over 'partitions' of our tables, and keep all rows in the output &#x2013; using the reserved word `OVER` we can introduce a *window* of our rows, and apply the function only over this 'window':



In [1]:
%%sql
SELECT    year,
          category,
          name,
          count() OVER (PARTITION by category) AS count
FROM      nobel_prizes
WHERE     year = 2013
ORDER BY  category

We can also give our windows names, using an alias:



In [1]:
%%sql
SELECT    year,
          category,
          name,
          count() OVER categories AS count
FROM      nobel_prizes
WHERE     year = 2013
WINDOW    categories AS (PARTITION by category)
ORDER BY  category

The aliased window definitions must come after any `WHERE` and `HAVING`, and before any `ORDER BY`.

So, if we use the reserved word `OVER` after a function, the function will be applied according to a 'window' (there is more to it than this, but this will suffice for now).
In the window we can:

-   define a partition, using `PARTITION BY`,

-   define an order, using `ORDER BY`, and

-   define a range, which we can use to define groups of rows relative to each other (but we won't look at ranges in the course).

The function will be applied to each partition, in the same way we applied aggregate functions on groups above, but now we won't collapse the partitions.
Observe that the partitioning and ordering are based only on the selection we make (i.e., only those rows which are chosen in our `WHERE` clause).

We can use our regular aggregate functions as window functions, but there are also a couple of dedicated window functions, such as (there are more, but we won't use them in the course):

-   `rank()`: ranks rows by the order specified in the window, ties can occur,

-   `row_number()`: as `rank()`, but now we avoid ties, and rank by row number in the output, and

-   `percent_rank()`: gives a value between 0.0 and 1.0 (so it's a bit of a misnomer), giving the row's relative rank within its partition.

You can find more [here](https://sqlite.org/windowfunctions.html).

Window functions can be very powerful, but we'll not delve too deeply into them in the course &#x2013; I want you to be aware of them, though!

**Problem:** Add one column which 'ranks' the laureates of 2013 in the table above according to the lengths of their names, within the categories &#x2013; shorter names should come before longer names.



In [1]:
%%sql

**Exercise:** For each laureate with the initial A, list their category, year, name, and 'freshness' within that category, i.e., the most recent laureate in a category is ranked 1, the second most recent laureate is ranked 2, etc. The ranks should be confined to laureates with the initial A.



In [1]:
%%sql

### Subqueries, Views and Common Table Expressions (CTEs)



We talked about subqueries the last time, here's just one 'refresher':

**Problem:** Show in which years there were prices in some category, but not in medicine.



In [1]:
%%sql

#### Correlated subqueries



Sometimes it's useful to have a subquery which is somehow dependent on the surrounding query.
As an example, solve the following problem:

**Problem:** List the names of all laureates who has the longest name of all laureates in their category (in case of ties, all should be listed) &#x2013; order by category.

Here we can use a subquery which is 'run' for each row in our main query:



In [1]:
%%sql
SELECT  category, year, name
FROM    nobel_prizes AS outer_nobel_prizes
WHERE   length(name) = (
            SELECT max(length(name))
            FROM   nobel_prizes
            WHERE  category = outer_nobel_prizes.category)
ORDER BY category

This is called a *correlated subquery* (since we refer to the enclosing query inside it).
We use an alias to distinguish between the `nobel_prize` table in the outer query and the `nobel_prize` table in the subquery (it's the same table, but we 'iterate' through it separately).

BTW, we could have skipped the `AS` in

    ...
    FROM    nobel AS outer_nobel
    ...

and just written:

    ...
    FROM    nobel outer_nobel
    ...

The general opinion is that we should use `AS`, as it makes it more obvious that we're defining an alias.



### When one table is not enough



In the preparation notes for this lecture, you were asked to think through the problem below.

---

**Problem:** We've been asked to write code to handle students and their college applications. Each student has a name, a student id (stil), and an average grade. The students apply for various majors at colleges, where each college has a name, is situated in a given state, and has a given enrollment. Each application is for a specific major at a specific college, and it can be accepted, rejected, or pending.

1.  What classes and/or data structures would you use to keep track of this data in Python or Java?
2.  How do you list all applications for a given student?
3.  How do you list all applications for a given major at a given college?
4.  Approximately how many lines of Python/Java code do you need to list the three programs (major@college) with the highest grade average for the 'last' accepted student (assuming the students are accepted in order of decreasing grade averages)?
5.  How would you persist your data, so it doesn't get lost if we have to restart the program?
6.  Can you come up with a way to let several programs access the data simultaneously?
7.  If you solve the previous problem, and want to allow several programs update the data simultaneously, how do you make sure the data isn't corrupted?

---

If we wanted to keep all the information above in one table, we'd need the following columns (below I use some column names which are in a style I don't normally use, but I think it could be good for you to be exposed to different styles):

-   student id (`s_id`)
-   student name (`s_name`)
-   average grade (`gpa`)
-   college (`c_name`)
-   state (`state`)
-   enrollment (`enrollment`)
-   major (`major`)
-   decision (`decision`)

Here is a (very small) example:

<div>
<table rules="all">
<tr>
  <th><code>s&lowbar;id</code></th>
  <th><code>s&lowbar;name</code></th>
  <th><code>gpa</code></th>
  <th><code>c&lowbar;name</code></th>
  <th><code>state</code></th>
  <th><code>enrollment</code></th>
  <th><code>major</code></th>
  <th><code>decision</code></th>
</tr>
<TR>
  <TD>123</TD><TD>Amy</TD><TD>3.9</TD>
  <TD>Stanford</TD><TD>CA</TD><TD>15000</TD>
  <TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>123</TD><TD>Amy</TD><TD>3.9</TD>
  <TD>Stanford</TD><TD>CA</TD><TD>15000</TD>
  <TD>EE</TD><TD>N</TD>
</TR>
<TR>
  <TD>123</TD><TD>Amy</TD><TD>3.9</TD>
  <TD>Berkeley</TD><TD>CA</TD><TD>36000</TD>
  <TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>123</TD><TD>Amy</TD><TD>3.9</TD>
  <TD>Cornell</TD><TD>NY</TD><TD>21000</TD>
  <TD>EE</TD><TD>Y</TD>
</TR>
<TR>
  <TD>234</TD><TD>Bob</TD><TD>3.6</TD>
  <TD>Berkeley</TD><TD>CA</TD><TD>36000</TD>
  <TD>biology</TD><TD>N</TD>
</TR>
<TR>
  <TD>345</TD><TD>Craig</TD><TD>3.5</TD>
  <TD>MIT</TD><TD>MA</TD><TD>10000</TD>
  <TD>bioengineering</TD><TD>Y</TD>
</TR>
<TR>
  <TD>345</TD><TD>Craig</TD><TD>3.5</TD>
  <TD>Cornell</TD><TD>NY</TD><TD>21000</TD>
  <TD>bioengineering</TD><TD>N</TD>
</TR>
<TR>
  <TD>345</TD><TD>Craig</TD><TD>3.5</TD>
  <TD>Cornell</TD><TD>NY</TD><TD>21000</TD>
  <TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>345</TD><TD>Craig</TD><TD>3.5</TD>
  <TD>Cornell</TD><TD>NY</TD><TD>21000</TD>
  <TD>EE</TD><TD>N</TD>
</TR>
<TR>
  <TD>678</TD><TD>Fay</TD><TD>3.8</TD>
  <TD>Stanford</TD><TD>CA</TD><TD>15000</TD>
  <TD>history</TD><TD>Y</TD>
</TR>
<TR>
  <TD>987</TD><TD>Helen</TD><TD>3.7</TD>
  <TD>Stanford</TD><TD>CA</TD><TD>15000</TD>
  <TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>987</TD><TD>Helen</TD><TD>3.7</TD>
  <TD>Berkeley</TD><TD>CA</TD><TD>36000</TD>
  <TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>876</TD><TD>Irene</TD><TD>3.9</TD>
  <TD>Stanford</TD><TD>CA</TD><TD>15000</TD>
  <TD>CS</TD><TD>N</TD>
</TR>
<TR>
  <TD>876</TD><TD>Irene</TD><TD>3.9</TD>
  <TD>MIT</TD><TD>MA</TD><TD>10000</TD>
  <TD>biology</TD><TD>Y</TD>
</TR>
<TR>
  <TD>876</TD><TD>Irene</TD><TD>3.9</TD>
  <TD>MIT</TD><TD>MA</TD><TD>10000</TD>
  <TD>marine biology</TD><TD>N</TD>
</TR>
<TR>
  <TD>765</TD><TD>Jay</TD><TD>2.9</TD>
  <TD>Stanford</TD><TD>CA</TD><TD>15000</TD>
  <TD>history</TD><TD>Y</TD>
</TR>
<TR>
  <TD>765</TD><TD>Jay</TD><TD>2.9</TD>
  <TD>Cornell</TD><TD>NY</TD><TD>21000</TD>
  <TD>history</TD><TD>N</TD>
</TR>
<TR>
  <TD>765</TD><TD>Jay</TD><TD>2.9</TD>
  <TD>Cornell</TD><TD>NY</TD><TD>21000</TD>
  <TD>psychology</TD><TD>Y</TD>
</TR>
<TR>
  <TD>543</TD><TD>Craig</TD><TD>3.4</TD>
  <TD>MIT</TD><TD>MA</TD><TD>10000</TD>
  <TD>CS</TD><TD>N</TD>
</TR>
</table></div>

but this table comes with some problems:

-   some of the information is repeated in many places (which is wasteful and error prone)

-   if we want to change Amy's gpa, we'd have to change it in several places (once per application she's made)

-   if a student withdraws her last application, we'd loose all information about that student (this has actually happened here already, as you'll see below)

In week 4 we'll discuss the theory behind these problems (functional dependencies), and see a technique we can use to find well behaved tables (normalization), but I hope that you intuitively can see that everything would be easier if we had the following three tables instead of the big table above:

-   one table, `students`, with information about the students (id, name and gpa)

<div>
<table rules="all">
<tr>
  <th><code>s&lowbar;id</code></th>
  <th><code>s&lowbar;name</code></th>
  <th><code>gpa</code></th>
</tr>

<TR><TD>123</TD>
<TD>Amy</TD>
<TD>3.9</TD>
</TR>
<TR><TD>234</TD>
<TD>Bob</TD>
<TD>3.6</TD>
</TR>
<TR><TD>345</TD>
<TD>Craig</TD>
<TD>3.5</TD>
</TR>
<TR><TD>456</TD>
<TD>Doris</TD>
<TD>3.9</TD>
</TR>
<TR><TD>543</TD>
<TD>Craig</TD>
<TD>3.4</TD>
</TR>
<TR><TD>567</TD>
<TD>Edward</TD>
<TD>2.9</TD>
</TR>
<TR><TD>654</TD>
<TD>Amy</TD>
<TD>3.9</TD>
</TR>
<TR><TD>678</TD>
<TD>Fay</TD>
<TD>3.8</TD>
</TR>
<TR><TD>765</TD>
<TD>Jay</TD>
<TD>2.9</TD>
</TR>
<TR><TD>789</TD>
<TD>Gary</TD>
<TD>3.4</TD>
</TR>
<TR><TD>876</TD>
<TD>Irene</TD>
<TD>3.9</TD>
</TR>
<TR><TD>987</TD>
<TD>Helen</TD>
<TD>3.7</TD>
</TR>
</table></div>

Here we have some students which weren't seen in the original table above, since they did not have any applications.



In [1]:
%%sql

-   one table, `colleges`, with college information (name and state)

<div>
<table rules="all">
<tr>
  <th><code>c&lowbar;name</code></th>
  <th><code>state</code></th>
  <th><code>enrollment</code></th>
</tr>
<TR><TD>Stanford</TD>
<TD>CA</TD>
<TD>15000</TD>
</TR>
<TR><TD>Berkeley</TD>
<TD>CA</TD>
<TD>36000</TD>
</TR>
<TR><TD>MIT</TD>
<TD>MA</TD>
<TD>10000</TD>
</TR>
<TR><TD>Cornell</TD>
<TD>NY</TD>
<TD>21000</TD>
</TR>
</table></div>



In [1]:
%%sql

-   one table, `applications`, with applications (student id, college name, major, and decision)

<div>
<table rules="all">
<tr>
  <th><code>s&lowbar;id</code></th>
  <th><code>c&lowbar;name</code></th>
  <th><code>major</code></th>
  <th><code>decision</code></th>
</tr>
<TR><TD>123</TD>
<TD>Stanford</TD>
<TD>CS</TD>
<TD>Y</TD>
</TR>
<TR><TD>123</TD>
<TD>Stanford</TD>
<TD>EE</TD>
<TD>N</TD>
</TR>
<TR><TD>123</TD>
<TD>Berkeley</TD>
<TD>CS</TD>
<TD>Y</TD>
</TR>
<TR><TD>123</TD>
<TD>Cornell</TD>
<TD>EE</TD>
<TD>Y</TD>
</TR>
<TR><TD>234</TD>
<TD>Berkeley</TD>
<TD>biology</TD>
<TD>N</TD>
</TR>
<TR><TD>345</TD>
<TD>MIT</TD>
<TD>bioengineering</TD>
<TD>Y</TD>
</TR>
<TR><TD>345</TD>
<TD>Cornell</TD>
<TD>bioengineering</TD>
<TD>N</TD>
</TR>
<TR><TD>345</TD>
<TD>Cornell</TD>
<TD>CS</TD>
<TD>Y</TD>
</TR>
<TR><TD>345</TD>
<TD>Cornell</TD>
<TD>EE</TD>
<TD>N</TD>
</TR>
<TR><TD>678</TD>
<TD>Stanford</TD>
<TD>history</TD>
<TD>Y</TD>
</TR>
<TR><TD>987</TD>
<TD>Stanford</TD>
<TD>CS</TD>
<TD>Y</TD>
</TR>
<TR><TD>987</TD>
<TD>Berkeley</TD>
<TD>CS</TD>
<TD>Y</TD>
</TR>
<TR><TD>876</TD>
<TD>Stanford</TD>
<TD>CS</TD>
<TD>N</TD>
</TR>
<TR><TD>876</TD>
<TD>MIT</TD>
<TD>biology</TD>
<TD>Y</TD>
</TR>
<TR><TD>876</TD>
<TD>MIT</TD>
<TD>marine biology</TD>
<TD>N</TD>
</TR>
<TR><TD>765</TD>
<TD>Stanford</TD>
<TD>history</TD>
<TD>Y</TD>
</TR>
<TR><TD>765</TD>
<TD>Cornell</TD>
<TD>history</TD>
<TD>N</TD>
</TR>
<TR><TD>765</TD>
<TD>Cornell</TD>
<TD>psychology</TD>
<TD>Y</TD>
</TR>
<TR><TD>543</TD>
<TD>MIT</TD>
<TD>CS</TD>
<TD>N</TD>
</TR>
</table></div>



In [1]:
%%sql

Nothing in these table is *redundant*, so if we use these tables, the issues we saw above goes away (we'll talk more about this in week 4).
But the benefits come with a cost, we now need a way to combine information from several tables at once.

Let's say we want to show the names and gpa of all students who has applied for CS at Stanford &#x2013; none of our three tables above contain all information we need, but if we start by looking in the applications table, we can get &#x2026;



In [1]:
%%sql
SELECT    s_id
FROM      applications
WHERE     c_name = 'Stanford' AND major = 'CS'

The best we can get here is the id of all relevant students, but given the id we can look in the `students` table for the name and gpa.

In regular programming languages, such as Python or Java, we could do that with functions such as `find_name(s_id)` and `find_gpa(s_id)`:



In [1]:
%%sql
SELECT    find_name(s_id), find_gpa(s_id)     -- not in SQL!
FROM      applications
WHERE     c_name = 'Stanford' AND major = 'CS'

But SQL is no regular language, it's based on relational algebra, and there we have another, really cool way of solving this problem: [*joins*](https://en.wikipedia.org/wiki/Join_(SQL)).



#### Inner joins



We solve the problem above by combining (*joining*) the `applications` table with the `students` table. There are several kinds of joins in SQL (and relational algebra), but the most straightforward kind is what's called an [*inner join*](https://en.wikipedia.org/wiki/Join_(SQL)#Inner_join).

The inner join of two tables is a new table, where rows from each table are joined together if the attributes of the rows matches a *join predicate* &#x2013; in our case we want to add student data to the application data where the student id in the applications table matches the student id in the students table.

This can be expressed as (see the [docs](https://sqlite.org/lang_select.html)):



In [1]:
%%sql
SELECT *
FROM   applications
       INNER JOIN students ON applications.s_id = students.s_id

UsageError: Cell magic `%%sql` not found.


and we would get the joined table (note that the first columns are from the `applications` table, whereas the last columns are from the `students` table &#x2013; also note that `s_id` and `s_id'` always is the same):

<div>
<table rules="all">
<tr>
  <th><code>s&lowbar;id</code></th>
  <th><code>c&lowbar;name</code></th>
  <th><code>major</code></th>
  <th><code>decision</code></th>
  <th><code>s&lowbar;id'</code></th>
  <th><code>s&lowbar;name</code></th>
  <th><code>gpa</code></th>
</tr>
<TR>
  <TD>123</TD><TD>Stanford</TD><TD>CS</TD>
  <TD>Y</TD><TD>123</TD><TD>Amy</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>123</TD><TD>Stanford</TD><TD>EE</TD>
  <TD>N</TD><TD>123</TD><TD>Amy</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>123</TD><TD>Berkeley</TD><TD>CS</TD>
  <TD>Y</TD><TD>123</TD><TD>Amy</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>123</TD><TD>Cornell</TD><TD>EE</TD>
  <TD>Y</TD><TD>123</TD><TD>Amy</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>234</TD><TD>Berkeley</TD><TD>biology</TD>
  <TD>N</TD><TD>234</TD><TD>Bob</TD><TD>3.6</TD>
</TR>
<TR>
  <TD>345</TD><TD>MIT</TD><TD>bioengineering</TD>
  <TD>Y</TD><TD>345</TD><TD>Craig</TD><TD>3.5</TD>
</TR>
<TR>
  <TD>345</TD><TD>Cornell</TD><TD>bioengineering</TD>
  <TD>N</TD><TD>345</TD><TD>Craig</TD><TD>3.5</TD>
</TR>
<TR>
  <TD>345</TD><TD>Cornell</TD><TD>CS</TD>
  <TD>Y</TD><TD>345</TD><TD>Craig</TD><TD>3.5</TD>
</TR>
<TR>
  <TD>345</TD><TD>Cornell</TD><TD>EE</TD>
  <TD>N</TD><TD>345</TD><TD>Craig</TD><TD>3.5</TD>
</TR>
<TR>
  <TD>678</TD><TD>Stanford</TD><TD>history</TD>
  <TD>Y</TD><TD>678</TD><TD>Fay</TD><TD>3.8</TD>
</TR>
<TR>
  <TD>987</TD><TD>Stanford</TD><TD>CS</TD>
  <TD>Y</TD><TD>987</TD><TD>Helen</TD><TD>3.7</TD>
</TR>
<TR>
  <TD>987</TD><TD>Berkeley</TD><TD>CS</TD>
  <TD>Y</TD><TD>987</TD><TD>Helen</TD><TD>3.7</TD>
</TR>
<TR>
  <TD>876</TD><TD>Stanford</TD><TD>CS</TD>
  <TD>N</TD><TD>876</TD><TD>Irene</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>876</TD><TD>MIT</TD><TD>biology</TD>
  <TD>Y</TD><TD>876</TD><TD>Irene</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>876</TD><TD>MIT</TD><TD>marine biology</TD>
  <TD>N</TD><TD>876</TD><TD>Irene</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>765</TD><TD>Stanford</TD><TD>history</TD>
  <TD>Y</TD><TD>765</TD><TD>Jay</TD><TD>2.9</TD>
</TR>
<TR>
  <TD>765</TD><TD>Cornell</TD><TD>history</TD>
  <TD>N</TD><TD>765</TD><TD>Jay</TD><TD>2.9</TD>
</TR>
<TR>
  <TD>765</TD><TD>Cornell</TD><TD>psychology</TD>
  <TD>Y</TD><TD>765</TD><TD>Jay</TD><TD>2.9</TD>
</TR>
<TR>
  <TD>543</TD><TD>MIT</TD><TD>CS</TD>
  <TD>N</TD><TD>543</TD><TD>Craig</TD><TD>3.4</TD>
</TR>
</table></div>

which contains all information we need.

We'll soon see other kinds of joins, but inner joins are the 'default', and so we can skip the reserved word `INNER`, and just write:



In [1]:
%%sql
SELECT *
FROM   applications
       JOIN students ON applications.s_id = students.s_id

Since the join predicate is just a test of equality of one attribute from each table (it's called an *equijoin*), and the attribute has the same name in both tables, we can use the simpler form:



In [1]:
%%sql
SELECT *
FROM   applications
       JOIN students USING (s_id)

which gives (observe that we now only have one <code>s&lowbar;id</code> column):

<div>
<table rules="all">
<tr>
  <th><code>s&lowbar;id</code></th>
  <th><code>c&lowbar;name</code></th>
  <th><code>major</code></th>
  <th><code>decision</code></th>
  <th><code>s&lowbar;name</code></th>
  <th><code>gpa</code></th>
</tr>
<TR>
  <TD>123</TD><TD>Stanford</TD><TD>CS</TD>
  <TD>Y</TD><TD>Amy</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>123</TD><TD>Stanford</TD><TD>EE</TD>
  <TD>N</TD><TD>Amy</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>123</TD><TD>Berkeley</TD><TD>CS</TD>
  <TD>Y</TD><TD>Amy</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>123</TD><TD>Cornell</TD><TD>EE</TD>
  <TD>Y</TD><TD>Amy</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>234</TD><TD>Berkeley</TD><TD>biology</TD>
  <TD>N</TD><TD>Bob</TD><TD>3.6</TD>
</TR>
<TR>
  <TD>345</TD><TD>MIT</TD><TD>bioengineering</TD>
  <TD>Y</TD><TD>Craig</TD><TD>3.5</TD>
</TR>
<TR>
  <TD>345</TD><TD>Cornell</TD><TD>bioengineering</TD>
  <TD>N</TD><TD>Craig</TD><TD>3.5</TD>
</TR>
<TR>
  <TD>345</TD><TD>Cornell</TD><TD>CS</TD>
  <TD>Y</TD><TD>Craig</TD><TD>3.5</TD>
</TR>
<TR>
  <TD>345</TD><TD>Cornell</TD><TD>EE</TD>
  <TD>N</TD><TD>Craig</TD><TD>3.5</TD>
</TR>
<TR>
  <TD>678</TD><TD>Stanford</TD><TD>history</TD>
  <TD>Y</TD><TD>Fay</TD><TD>3.8</TD>
</TR>
<TR>
  <TD>987</TD><TD>Stanford</TD><TD>CS</TD>
  <TD>Y</TD><TD>Helen</TD><TD>3.7</TD>
</TR>
<TR>
  <TD>987</TD><TD>Berkeley</TD><TD>CS</TD>
  <TD>Y</TD><TD>Helen</TD><TD>3.7</TD>
</TR>
<TR>
  <TD>876</TD><TD>Stanford</TD><TD>CS</TD>
  <TD>N</TD><TD>Irene</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>876</TD><TD>MIT</TD><TD>biology</TD>
  <TD>Y</TD><TD>Irene</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>876</TD><TD>MIT</TD><TD>marine biology</TD>
  <TD>N</TD><TD>Irene</TD><TD>3.9</TD>
</TR>
<TR>
  <TD>765</TD><TD>Stanford</TD><TD>history</TD>
  <TD>Y</TD><TD>Jay</TD><TD>2.9</TD>
</TR>
<TR>
  <TD>765</TD><TD>Cornell</TD><TD>history</TD>
  <TD>N</TD><TD>Jay</TD><TD>2.9</TD>
</TR>
<TR>
  <TD>765</TD><TD>Cornell</TD><TD>psychology</TD>
  <TD>Y</TD><TD>Jay</TD><TD>2.9</TD>
</TR>
<TR>
  <TD>543</TD><TD>MIT</TD><TD>CS</TD>
  <TD>N</TD><TD>Craig</TD><TD>3.4</TD>
</TR>
</table></div>

In the first join above we got two `s_id` columns (SQL can't possibly know that `s_id` describes the same thing in both tables), but with the `JOIN` `USING` construct, we tell SQL that they describe the same thing, so the join returns only one column for `s_id`.

In general, we can compare sets of several attributes in a `JOIN` `USING` query.

*Observe that a row in one table could match several rows in the other table, and it would therefore generate several rows in the combined table.*

Once we've joined our tables, we can proceed as we did before, with a `WHERE` clause:



In [1]:
%%sql
SELECT    s_name, gpa
FROM      applications
          JOIN students USING (s_id)
WHERE     c_name = 'Stanford' AND major = 'CS'

If we look at the syntax diagram for the [`SELECT` statement](https://sqlite.org/lang_select.html), we see that we join our tables in the `FROM` clause (see `join-clause`), and that we can use everything we've learned so far to work on the joined tables (so we can group and use window functions and subqueries just as we've done before).

The combined, rather clumsy table will probably never be created, database engines are *extremely* clever when they decide on what data they need to keep around to execute their queries &#x2013; so we normally do not need to worry much about efficiency (we'll talk a little more about this later in the course).

---

It could be useful to stop here for a while, and consider how you'd solve the problem above using Python or Java (or any other regular programming language).
In case we let the students keep track of their applications, we'd need to loop through all students, and then all their applications looking for applications to CS@Stanford.

Deciding on who should keep track of what is a typical example of premature optimization, and using a relational database relieves us of this problem.

---

**Problem:** Show the id and name for all students who has applied for CS in California (state code `CA`).



In [1]:
%%sql

#### Cross joins



As we saw above, an inner join combines rows in two tables when the rows matches on some join predicate.
The predicate is typically dependent on column values in both rows (e.g., that the student ids in each table should match), but we can also use a seemingly nonsensical predicate such as `true` (which is always true):



In [1]:
%%sql
SELECT   *
FROM     applications
         JOIN students ON true

This would mean that all rows in the applications table would match all rows in the students table, and we would get what's called the *cartesian product* of the two tables (or sets of rows), i.e., all combinations of rows in the tables.

This is exactly what a cross join is &#x2013; we can write it as:



In [1]:
%%sql
SELECT   *
FROM     applications
         CROSS JOIN students

and we typically use a `WHERE` clause to narrow the generated table into a table where only matching rows remains afterwards.

So, the query



In [1]:
%%sql
SELECT    s_name, gpa
FROM      applications
          JOIN students USING (s_id)
WHERE     c_name = 'Stanford' AND major = 'CS'

could have been written using a cross join instead:



In [1]:
%%sql
SELECT    s_name, gpa
FROM      applications
          CROSS JOIN students
WHERE     applications.s_id = students.s_id
          AND c_name = 'Stanford'
          AND major = 'CS'

This may look terrible at first sight, creating a huge table and then throwing away most of the combined rows, but, as said above, database engines are very, very clever, and they might find a way to make even this query fast.
But I'd still recommend that you use inner queries, since it might help the database to find a faster query plan, and it's conceptually easier to understand for a person looking at your code.

By the way, there is a shortcut for cross joins, &#x2026;



In [1]:
%%sql
SELECT    s_name, gpa
FROM      applications, students
WHERE     applications.s_id = students.s_id
          AND c_name = 'Stanford'
          AND major = 'CS'

but, once again, opt for inner joins instead!



#### Outer joins



We saw above that a row in one table which matches several rows in the other table would give rise to several rows in the combined table.
But it also goes the other way &#x2013; when we do an inner join on two tables, rows for which the join predicate doesn't match any row in the other table will 'disappear' in the combined table.

So if we wanted to list all students and their applications, and wrote:



In [1]:
%%sql
SELECT    *
FROM      students
          JOIN applications USING (s_id)
ORDER BY  s_name

we would get all applications, but those students who hadn't applied for any major would be left out.

If we want to make sure that all rows in the first ('left') table survives the join, we can use a *left outer join*.
On outer join is a join in which we guarantee that all rows in one or both our tables will be present in the result.
But that leads to an important question: how do we combine an 'uncombinable' row?
The answer is that we combine it with a row full of `NULL` values in the other table &#x2013; if we try a left outer join, and puts the `students` table first (on the left), we get:



In [1]:
%%sql
SELECT    *
FROM      students
          LEFT OUTER JOIN applications USING (s_id)
ORDER BY  s_name

<div>
<table rules="all">
<tr>
  <th><code>s&lowbar;id</code></th>
  <th><code>c&lowbar;name</code></th>
  <th><code>major</code></th>
  <th><code>decision</code></th>
  <th><code>s&lowbar;name</code></th>
  <th><code>gpa</code></th>
</tr>
<TR>
  <TD>123</TD><TD>Amy</TD><TD>3.9</TD>
  <TD>Berkeley</TD><TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>123</TD><TD>Amy</TD><TD>3.9</TD>
  <TD>Cornell</TD><TD>EE</TD><TD>Y</TD>
</TR>
<TR>
  <TD>123</TD><TD>Amy</TD><TD>3.9</TD>
  <TD>Stanford</TD><TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>123</TD><TD>Amy</TD><TD>3.9</TD>
  <TD>Stanford</TD><TD>EE</TD><TD>N</TD>
</TR>
<TR>
  <TD>654</TD><TD>Amy</TD><TD>3.9</TD>
  <TD><code>NULL</code></TD>
  <TD><code>NULL</code></TD>
  <TD><code>NULL</code></TD>
</TR>
<TR>
  <TD>234</TD><TD>Bob</TD><TD>3.6</TD>
  <TD>Berkeley</TD><TD>biology</TD><TD>N</TD>
</TR>
<TR>
  <TD>345</TD><TD>Craig</TD><TD>3.5</TD>
  <TD>Cornell</TD><TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>345</TD><TD>Craig</TD><TD>3.5</TD>
  <TD>Cornell</TD><TD>EE</TD><TD>N</TD>
</TR>
<TR>
  <TD>345</TD><TD>Craig</TD><TD>3.5</TD>
  <TD>Cornell</TD><TD>bioengineering</TD><TD>N</TD>
</TR>
<TR>
  <TD>345</TD><TD>Craig</TD><TD>3.5</TD>
  <TD>MIT</TD><TD>bioengineering</TD><TD>Y</TD>
</TR>
<TR>
  <TD>543</TD><TD>Craig</TD><TD>3.4</TD>
  <TD>MIT</TD><TD>CS</TD><TD>N</TD>
</TR>
<TR>
  <TD>456</TD><TD>Doris</TD><TD>3.9</TD>
  <TD><code>NULL</code></TD>
  <TD><code>NULL</code></TD>
  <TD><code>NULL</code></TD>
</TR>
<TR>
  <TD>567</TD><TD>Edward</TD><TD>2.9</TD>
  <TD><code>NULL</code></TD>
  <TD><code>NULL</code></TD>
  <TD><code>NULL</code></TD>
</TR>
<TR>
  <TD>678</TD><TD>Fay</TD><TD>3.8</TD>
  <TD>Stanford</TD><TD>history</TD><TD>Y</TD>
</TR>
<TR>
  <TD>789</TD><TD>Gary</TD><TD>3.4</TD>
  <TD><code>NULL</code></TD>
  <TD><code>NULL</code></TD>
  <TD><code>NULL</code></TD>
</TR>
<TR>
  <TD>987</TD><TD>Helen</TD><TD>3.7</TD>
  <TD>Berkeley</TD><TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>987</TD><TD>Helen</TD><TD>3.7</TD>
  <TD>Stanford</TD><TD>CS</TD><TD>Y</TD>
</TR>
<TR>
  <TD>876</TD><TD>Irene</TD><TD>3.9</TD>
  <TD>MIT</TD><TD>biology</TD><TD>Y</TD>
</TR>
<TR>
  <TD>876</TD><TD>Irene</TD><TD>3.9</TD>
  <TD>MIT</TD><TD>marine biology</TD><TD>N</TD>
</TR>
<TR>
  <TD>876</TD><TD>Irene</TD><TD>3.9</TD>
  <TD>Stanford</TD><TD>CS</TD><TD>N</TD>
</TR>
<TR>
  <TD>765</TD><TD>Jay</TD><TD>2.9</TD>
  <TD>Cornell</TD><TD>history</TD><TD>N</TD>
</TR>
<TR>
  <TD>765</TD><TD>Jay</TD><TD>2.9</TD>
  <TD>Cornell</TD><TD>psychology</TD><TD>Y</TD>
</TR>
<TR>
  <TD>765</TD><TD>Jay</TD><TD>2.9</TD>
  <TD>Stanford</TD><TD>history</TD><TD>Y</TD>
</TR>
</table></div>

Here we see 'the other Amy' (i.e., the one with `s_id` 654), Doris, Edward, and Gary, who apparently haven't applied for anything &#x2013; the columns which would have been taken from the `applications` table in case of a match are now set to `NULL`.

The fact that an outer join pads rows which can't be matched in the 'other' table with `NULL` values gives us a nice way to find out which values might be missing in a table &#x2013; se the following problem.

**Problem:** We want to find all students who hasn't applied for any major &#x2013; do it in two ways:

-   Using a subquery.



In [1]:
%%sql

-   Using an outer join (checking where we've padded with `NULL` columns).



In [1]:
%%sql

This way of finding missing values may look like a hack, but it's in fact idiomatic.



##### Other outer joins



There are three kinds of outer joins:

-   `LEFT OUTER JOIN`, which can be abbreviated `LEFT JOIN` &#x2013; this is when we keep all rows in the first (leftmost) table, padding with `NULL` values when they don't match any row in the other table.

-   `RIGHT OUTER JOIN`, which can be abbreviated `RIGHT JOIN` &#x2013; this is when we keep all rows in the second (rightmost) table, padding with `NULL` values when they don't match any row in the other table.

-   `FULL OUTER JOIN`, which can be abbreviated `FULL JOIN` &#x2013; this is when we keep all rows in both tables, padding with `NULL` values when they don't match any row in the other table.



#### Exercises



**Exercise:** How many students with a gpa above 3.5 have applied for a CS major at a college in California (CA)?



In [1]:
%%sql

**Hint:** It could be a good idea to first define a CTE with the relevant students.

Your query should return 3.

**Exercise:** List the id, name, and gpa for all students who have applied for more than one major in California (CA).



In [1]:
%%sql

Your query should return:

    s_id  s_name  gpa
    ----  ------  ---
    123   Amy     3.9
    987   Helen   3.7

**Exercise:** Which are the three programs (major@college) with the lowest gpa for any accepted student.



In [1]:
%%sql

The output should be:

    c_name    major       min(gpa)
    --------  ----------  --------
    Cornell   psychology  2.9
    Stanford  history     2.9
    Cornell   CS          3.5

And which are the three programs with the highest gpa for a student not accepted?



In [1]:
%%sql

The output should be:

    c_name    major           max(gpa)
    --------  --------------  --------
    MIT       marine biology  3.9
    Stanford  CS              3.9
    Stanford  EE              3.9

**Comment:** Apparently the gpa isn't the only criterion for acceptance, since Irene (`s_id` 876) wasn't accepted at CS@Stanford, but Helen (`s_id` 987) was, although Irene has a higher gpa.



### Set operations



Relational databases and SQL are based on relational algebra, and this means that we can use simple set operations on our data.
If we look at the `compound-operator` in the [`SELECT` statement](https://sqlite.org/lang_select.html), we see that we can combine the results of queries using `UNION`, `INTERSECT`, and `EXCEPT`.

This only works if the queries return exactly the same columns.

<center>
<img src="./select-compound-01.svg">
</center>

**Problem:** List the names and gpa for students who have applied for a major at a college in California, but haven't applied in any other state.



In [1]:
%%sql

### INSERT, UPDATE and DELETE



To maintain our databases, we need ways to change their contents, and for that we have tree important functions:

-   [`INSERT`](https://www.sqlite.org/lang_insert.html): inserts a row into a table

-   [`UPDATE`](https://www.sqlite.org/lang_update.html): updates rows which matches some predicate

-   [`DELETE`](https://www.sqlite.org/lang_delete.html): deletes rows which matches some predicate

We'll talk more about these functions next week, when we talk about modeling (then we'll also discuss the concept of *keys*, which is quite important when we modify our tables).

